In [1]:
# -*- coding: utf-8 -*-
import json
import os
import pandas as pd
import cv2
from PIL import Image
from shutil import copyfile

base_path = '/home/wzx/wzx_frcnn/Dataset/Open Images Dataset v5 (Bounding Boxes)'
json_path = os.path.join(base_path,'jimu')

In [2]:
# Save images to train and test directory
#定义train 和 test 的目录
train_path = os.path.join(base_path, 'train')
test_path = os.path.join(base_path, 'test')

In [3]:
def newfile(path):
    # 判断路径是否存在
    isExists=os.path.exists(path)
    # 不存在
    if not isExists:
    # 创建目录操作函数
        os.mkdir(path)
        print(path+' 创建成功')
    #存在
    else:
        print(path+' 目录已存在')

In [4]:
# 调用函数
newfile(train_path)
newfile(test_path)  
# Copy train images to train directory
train_imgs = os.listdir(os.path.join(json_path, 'train'))
train_imgs = [f for f in train_imgs if f.endswith('g')]
print(train_imgs[2])

for j in range(len(train_imgs)):
    original_path = os.path.join(os.path.join(json_path,'train'), train_imgs[j])
    new_path = os.path.join(train_path,train_imgs[j])
    copyfile(original_path, new_path)
    
# Copy test images to test directory
'''    for j in range(len(test_imgs)):
        original_path = os.path.join(os.path.join(base_path, classes[i]), test_imgs[j])
        new_path = os.path.join(test_path, test_imgs[j])
        copyfile(original_path, new_path)'''

# print the number of train_imgs & test_imgs
print('number of training images: ', len(os.listdir(train_path))) 
# subtract one because there is one hidden file named '.DS_Store'
print('number of test images: ', len(os.listdir(test_path)))

/home/wzx/wzx_frcnn/Dataset/Open Images Dataset v5 (Bounding Boxes)/train 创建成功
/home/wzx/wzx_frcnn/Dataset/Open Images Dataset v5 (Bounding Boxes)/test 创建成功
rs-save-to-disk-output-Color8.png
number of training images:  30
number of test images:  0


In [5]:
label_names = ['COL','CON','CUB','TRI']
train_df = pd.DataFrame(columns=['FileName', 'XMin', 'XMax', 'YMin', 'YMax', 'ClassName'])
for filename in os.listdir(os.path.join(json_path,'train')):
    # 定义特征点
    x1 ,y1 = 0, 0   # 框左上  x: 距左，y:距上
    x2, y2 = 0, 0   # 框右下
    mark_x1, mark_y1 = 0, 0    # 左上点
    mark_x2, mark_y2 = 0, 0    # 右上点
    mark_x3, mark_y3 = 0, 0    # 左下点
    mark_x4, mark_y4 = 0, 0    # 右下点
    if filename.endswith(".json"):
        data = json.load(open(os.path.join(os.path.join(json_path,'train'),filename), 'r'))
        img_name = data['imagePath'] # 拿到图像的名字

        for obj in data['shapes']:
            if obj['shape_type'] != "rectangle":
                continue
            if obj['label'] in label_names:   # label_name in classes
                class_name = obj['label']
                XMin = obj['points'][0][0]
                YMin = obj['points'][0][1]
                XMax = obj['points'][1][0]
                YMax = obj['points'][1][1]
        
            #write into df
            train_df = train_df.append({'FileName': img_name, 
                                        'XMin': XMin, 
                                        'XMax': XMax, 
                                        'YMin': YMin, 
                                        'YMax': YMax, 
                                        'ClassName': class_name}, 
                                       ignore_index=True)

In [6]:
'''

test_df = pd.DataFrame(columns=['FileName', 'XMin', 'XMax', 'YMin', 'YMax', 'ClassName'])

# Find boxes in each image and put them in a dataframe
test_imgs = os.listdir(test_path)
test_imgs = [name for name in test_imgs if not name.startswith('.')]

for i in range(len(test_imgs)):
    sys.stdout.write('Parse test_imgs ' + str(i) + '; Number of boxes: ' + str(len(test_df)) + '\r')
    sys.stdout.flush()
    img_name = test_imgs[i]
    img_id = img_name[0:16]
    tmp_df = annotations_bbox[annotations_bbox['ImageID']==img_id]
    for index, row in tmp_df.iterrows():
        labelName = row['LabelName']
        for i in range(len(label_names)):
            if labelName == label_names[i]:
                val_df = val_df.append({'FileName': img_name, 
                                            'XMin': row['XMin'], 
                                            'XMax': row['XMax'], 
                                            'YMin': row['YMin'], 
                                            'YMax': row['YMax'], 
                                            'ClassName': classes[i]}, 
                                           ignore_index=True)
'''

"\n\ntest_df = pd.DataFrame(columns=['FileName', 'XMin', 'XMax', 'YMin', 'YMax', 'ClassName'])\n\n# Find boxes in each image and put them in a dataframe\ntest_imgs = os.listdir(test_path)\ntest_imgs = [name for name in test_imgs if not name.startswith('.')]\n\nfor i in range(len(test_imgs)):\n    sys.stdout.write('Parse test_imgs ' + str(i) + '; Number of boxes: ' + str(len(test_df)) + '\r')\n    sys.stdout.flush()\n    img_name = test_imgs[i]\n    img_id = img_name[0:16]\n    tmp_df = annotations_bbox[annotations_bbox['ImageID']==img_id]\n    for index, row in tmp_df.iterrows():\n        labelName = row['LabelName']\n        for i in range(len(label_names)):\n            if labelName == label_names[i]:\n                val_df = val_df.append({'FileName': img_name, \n                                            'XMin': row['XMin'], \n                                            'XMax': row['XMax'], \n                                            'YMin': row['YMin'], \n                     

In [7]:
train_df.to_csv(os.path.join(base_path, 'train.csv'))
# test_df.to_csv(os.path.join(base_path, 'test.csv'))

In [8]:
print(train_df.head())
train_df = pd.read_csv(os.path.join(base_path, 'train.csv'))

# For training
f= open(base_path + "/annotation.txt","w+")
for idx, row in train_df.iterrows():
#     sys.stdout.write(str(idx) + '\r')
#     sys.stdout.flush()
    img = cv2.imread((base_path + '/train/' + row['FileName']))
    height, width = img.shape[:2]
    x1 = int(row['XMin'] * width)
    x2 = int(row['XMax'] * width)
    y1 = int(row['YMin'] * height)
    y2 = int(row['YMax'] * height)
    
    google_colab_file_path = '/home/wzx/wzx_frcnn/Dataset/Open Images Dataset v5 (Bounding Boxes)/train'
    fileName = os.path.join(google_colab_file_path, row['FileName'])
    className = row['ClassName']
    f.write(fileName + ',' + str(x1) + ',' + str(y1) + ',' + str(x2) + ',' + str(y2) + ',' + className + '\n')
f.close()

                             FileName        XMin        XMax        YMin  \
0  rs-save-to-disk-output-Color20.png  723.729730  789.945946  348.324324   
1  rs-save-to-disk-output-Color20.png  677.783784  768.324324  476.702703   
2  rs-save-to-disk-output-Color20.png  508.864865  614.270270  502.378378   
3  rs-save-to-disk-output-Color20.png  575.081081  709.000000  675.351351   
4  rs-save-to-disk-output-Color20.png  121.027027  206.000000  619.945946   

         YMax ClassName  
0  410.486486       CUB  
1  565.891892       CON  
2  596.972973       COL  
3  719.000000       CUB  
4  719.000000       COL  
